# Codes here verify the equation: x\*w = x(\*)w' = ifft(fft(x).*fft(w'))
# x is the input singal
# w is the convolutional kernel
# * is ths conv operation in deep learning
# (*) is the conv in signal processing 
# .* is the element wise multiply
# fft is Fourier transform
# ifft is inverse fourier transformation
# w' is flip of w

In [1]:
###########################

# The first part is generat x and w

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F



# buidng two singal
input_signal_length = 6
kernel_filter_length =4

#generate signal x
X = np.random.rand(input_signal_length)

#generate weight in a filter of 1D CNN
w = np.random.rand(kernel_filter_length)

print("the singal X:\n\n",X, end="\n\n")
print("the kernal W:\n\n", w, end="\n\n")

# expend dimention of signal to fit requirement of CNN
X_torch = torch.from_numpy(X).unsqueeze(0).unsqueeze(0)

# expend dimention of filter to fit requirement of CNN
w_torch = torch.from_numpy(w).unsqueeze(0).unsqueeze(0)



the singal X:

 [0.9037642  0.16311098 0.05093814 0.29195908 0.76054364 0.24063323]

the kernal W:

 [0.89583342 0.53699392 0.8601418  0.74147953]



# result of x*w

In [3]:
# the conventional reslut of deep learning
m = nn.ConstantPad1d(kernel_filter_length-1, 0)
deeplearning_conv_result = F.conv1d(m(X_torch) ,w_torch)
print('result of 1D convolution in deep learning: \n\n x*w = \n\n',deeplearning_conv_result.numpy(),'\n')

result of 1D convolution in deep learning: 

 x*w = 

 [[[0.67012266 0.89830881 0.66338403 1.15750748 0.98852749 1.03501233
   0.87693272 0.810539   0.21556729]]] 



# verify: x(\*)w' = x\*w

In [4]:
###########
#  this is important:
#  conventional in signal processing for x and y_reverse(y') equals to y
###########
w_reverse = np.flip(w,axis=0)
print('kernel in signal processing w\':\n\n',w_reverse, end="\n\n")



# the conventional reslut of signal processing
signal_processing_result = np.convolve(X, w_reverse, mode='full')
print('result of 1D convolution in signal processing: \n\n x(*)w\'= \n\n',signal_processing_result)
print('\n\nthe result is same with the x*w = \n\n', deeplearning_conv_result.numpy())


kernel in signal processing w':

 [0.74147953 0.8601418  0.53699392 0.89583342]

result of 1D convolution in signal processing: 

 x(*)w'= 

 [0.67012266 0.89830881 0.66338403 1.15750748 0.98852749 1.03501233
 0.87693272 0.810539   0.21556729]


the result is same with the x*w = 

 [[[0.67012266 0.89830881 0.66338403 1.15750748 0.98852749 1.03501233
   0.87693272 0.810539   0.21556729]]]


# We need zero padding X and W' to same length for doing fft(x).*fft(w')
# the cell below will get X_pad and W'_pad

In [5]:
print("the singal X:\n\n",X, end="\n\n")

X_pad = np.pad(X, (0,len(w_reverse)-1), 'constant', constant_values=(0, 0))

print("the singal X_pad:\n\n",X_pad, end="\n\n")


print("the kernal W:\n\n", w, end="\n\n")
w_reverse_pad = np.pad(w_reverse, (0,len(X)-1), 'constant', constant_values=(0, 0))

print("the kernal W\'_pad:\n\n", w_reverse_pad, end="\n\n")




the singal X:

 [0.9037642  0.16311098 0.05093814 0.29195908 0.76054364 0.24063323]

the singal X_pad:

 [0.9037642  0.16311098 0.05093814 0.29195908 0.76054364 0.24063323
 0.         0.         0.        ]

the kernal W:

 [0.89583342 0.53699392 0.8601418  0.74147953]

the kernal W'_pad:

 [0.74147953 0.8601418  0.53699392 0.89583342 0.         0.
 0.         0.         0.        ]



# x\*w = ifft(fft(x_pad).*fft(w'_pad))

In [6]:
X_pad_fft = np.fft.fft(X_pad)
w_reverse_pad_fft = np.fft.fft(w_reverse_pad)
print("singal in frequency domain: \n\n fft(x_pad) = \n\n",X_pad_fft, end="\n\n")
print("kernel in frequency domain: \n\n fft(w\'_pad) = \n\n",w_reverse_pad_fft, end="\n\n")

result = np.multiply(X_pad_fft,w_reverse_pad_fft)
print("conv result in frequency domain: \n\n fft(x_pad).fft(w'_pad) = \n\n",result, end="\n\n")

print("reconstruct result from frequency domain result: \n\n ifft(fft(x_pad).fft(w'_pad) )= \n\n",np.fft.ifft(result), end="\n\n")
print('\n\nthe result is same with the \n\n x*w = \n\n', deeplearning_conv_result.numpy())


singal in frequency domain: 

 fft(x_pad) = 

 [ 2.41094927+0.j         -0.04921829-0.5856738j   1.50518837+0.40898113j
  0.58811029-0.54740015j  0.8173839 +0.23612299j  0.8173839 -0.23612299j
  0.58811029+0.54740015j  1.50518837-0.40898113j -0.04921829+0.5856738j ]

kernel in frequency domain: 

 fft(w'_pad) = 

 [ 3.03444867+0.j          1.04571768-1.85753877j -0.06168435-0.25492255j
  0.9387451 -0.27985427j -0.10334487-0.72482728j -0.10334487+0.72482728j
  0.9387451 +0.27985427j -0.06168435+0.25492255j  1.04571768+1.85753877j]

conv result in frequency domain: 

 fft(x_pad).fft(w'_pad) = 

 [ 7.31590181+0.j         -1.13938023-0.52102457j  0.01141195-0.40893419j
  0.39889338-0.67845438j  0.08667595-0.61686425j  0.08667595+0.61686425j
  0.39889338+0.67845438j  0.01141195+0.40893419j -1.13938023+0.52102457j]

reconstruct result from frequency domain result: 

 ifft(fft(x_pad).fft(w'_pad) )= 

 [0.67012266-1.23358114e-17j 0.89830881-4.93432455e-17j
 0.66338403-4.93432455e-17j 1.1575074

# above result might seems different this is because when computer calculate FFT is have some remainders, thus bring a very small complex  number 

# below is the equation in our paper

In [7]:
# varification of singal processing conv equation

def maumal_conv(x,y,n):
    N = len(x)
    M = len(y)
    result = 0
    for m in range(M):
        if n-M+1+m <N and n-M+1+m>= 0:
            temp = x[n-M+1+m]
        else:
            temp = 0.0
        result = result+ temp*y[M-1-m]
    return result
        


signal_processing_result = np.convolve(X, w_reverse, mode='full')

print("correct result:\n\n",signal_processing_result,end='\n\n')

print("result calculated by the equation in the paper:\n\n")
for i in range(input_signal_length+kernel_filter_length-1):
    print(maumal_conv(X,w_reverse,i),end='\t')


correct result:

 [0.67012266 0.89830881 0.66338403 1.15750748 0.98852749 1.03501233
 0.87693272 0.810539   0.21556729]

result calculated by the equation in the paper:


0.670122656166945	0.8983088139564156	0.6633840339807204	1.157507483867864	0.988527489182509	1.0350123316819304	0.8769327167326233	0.8105389966910408	0.2155672916434485	